In [29]:
import pandas as pd
import json

from pydantic import BaseModel
from typing import Optional
from datetime import datetime

In [30]:
df = pd.read_json('resources/result.json')

In [ ]:
# Global parameters
TARGET_CHAT_NAME = ''
TARGET_IMITATION_SOURCE_NAME = ''

In [32]:
chats = df['chats']['list']
chat_1 = next((chat for chat in chats if chat.get('name') == TARGET_CHAT_NAME), None)

if chat_1:
    original_message_count = len(chat_1.get("messages", []))

    filtered_messages = [
        msg for msg in chat_1.get("messages", [])
        if "file" not in msg and "forwarded_from" not in msg
    ]

    filtered_message_count = len(filtered_messages)

    chat_1["messages"] = filtered_messages

    chat_display = {k: v for k, v in chat_1.items() if k != 'messages'}
    chat_display["original_message_count"] = original_message_count
    chat_display["filtered_message_count"] = filtered_message_count

    print(f"Chat <{TARGET_CHAT_NAME}>: {json.dumps(chat_display, ensure_ascii=False, indent=2)}")
else:
    print(f"Chat <{TARGET_CHAT_NAME}> not found.")


Chat <Арман>: {
  "name": "Арман",
  "type": "personal_chat",
  "id": 765069481,
  "original_message_count": 35063,
  "filtered_message_count": 32319
}


In [33]:
from pydantic import BaseModel
from typing import Optional
from datetime import datetime
import json

class InstructionFormat(BaseModel):
    instruction: str
    input: Optional[str] = None
    output: str

messages = chat_1["messages"]

instructions = []
for i, msg in enumerate(messages):
    if msg.get("from") == TARGET_IMITATION_SOURCE_NAME:
        previous_msg = messages[i - 1]["text"] if i > 0 and "text" in messages[i - 1] else None

        instruction_data = {
            "instruction": "Respond based on the conversation context.",
            "input": previous_msg,
            "output": msg["text"],
        }
        instructions.append(instruction_data)

with open("fine_tune_data.json", "w", encoding="utf-8") as f:
    json.dump(instructions, f, ensure_ascii=False, indent=2)